In [ ]:
!(@isdefined cleanup_hook) && IJulia.push_postexecute_hook(() -> (empty!(In); empty!(Out); GC.gc(true))); cleanup_hook = true;

In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.cluster as cluster

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
# rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")
include("../../../functions/utils.jl")

In [ ]:
experiment_filename_1 = "20220407_090156"
server_1 = 5

experiment_filename_2 = "20220407_104712"
server_2 = 5

experimenter = "jen"
analyzer = "chuyu"

In [ ]:
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

In [ ]:
NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global Z_all = HDF5.readmmap(NMF_file["Z_all"])
global X_all = HDF5.readmmap(NMF_file["X_all"])
global Y_all = HDF5.readmmap(NMF_file["Y_all"])
global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
global A_dFF_1 = HDF5.readmmap(NMF_file["A_dFF"]); # bs update
global A_baseline_1 = HDF5.readmmap(NMF_file["A_baseline"]); # bs update
close(NMF_file)

# for multi sessions
NMF_filename = joinpath(data_path(ds_save_cy_2), "NMF_merge.h5") # bs update
NMF_file = h5open(NMF_filename, "r") # bs update
global A_dFF_2 = HDF5.readmmap(NMF_file["A_dFF"]); # bs update
global A_baseline_2 = HDF5.readmmap(NMF_file["A_baseline"]); # bs update
close(NMF_file) # bs update

n_neuron = length(X_all);

In [ ]:
# whether individual roi belongs to a certain region
region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
region_bool_file = h5open(region_bool_filename, "r")
global region_names = read(region_bool_file, "region_names")
global region_roi_bool = read(region_bool_file, "region_roi_bool")
close(region_bool_file)

In [ ]:
# for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
whether_tel = falses(n_neuron)
for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
    if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
        whether_tel[which_neuron] = true
    end
end

In [ ]:
all_files = readdir(data_path(ds_save_cy_1))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*!occursin("shuffleall", candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(info_filename, "r")

place_cell_index_1 = HDF5.readmmap(file["place_cell_index"])
close(file)
    

In [ ]:
all_files = readdir(joinpath(data_path(ds_save_cy_1) ,"oblique_corrected"))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*!occursin("shuffleall", candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(data_path(ds_save_cy_1), "oblique_corrected",save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
specificity_1 = HDF5.readmmap(file["specificity"])
valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
entropy_1 = HDF5.readmmap(file["entropy_all"])
close(file)
    

In [ ]:
plot_quantity(specificity_1[valid_roi_1]; label= "specifity",vmax=0.1, which_neuron=valid_roi_1)

In [ ]:
plot_loc(place_cell_index_1; valid_neurons=intersect(valid_roi_1), label= length(place_cell_index_1))

In [ ]:
tel_place_cell_index_1 = intersect(place_cell_index_1, findall(whether_tel))
plot_loc(tel_place_cell_index_1; valid_neurons=intersect(valid_roi_1), label= length(tel_place_cell_index_1))

In [ ]:
place_cell_index = union(place_cell_index_1)

plot_loc(place_cell_index; valid_neurons=intersect(valid_roi_1), label= length(place_cell_index))

tel_place_cell_index = union(tel_place_cell_index_1) #what we should use in following analysis

plot_loc(tel_place_cell_index; valid_neurons=intersect(valid_roi_1), label= length(tel_place_cell_index))

In [ ]:
ds_save_cy_roi = Dataset("20220406_111526", "chuyu", gethostname() == "roli-$(9)" ? "/data" : "/nfs/data$(9)")
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_roi), "old/orientation_correction.h5"))
chamber_roi = read(orientation_correction_file,"chamber_roi_r")
close(orientation_correction_file)

chamber_roi[chamber_roi.!=0].=1;

In [ ]:
# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
countour_array_1 = read(orientation_correction_file,"countour_array")
center_loc_1 = read(orientation_correction_file,"center_loc")
close(orientation_correction_file)


img_bg_1 = h5open(ds_save_1, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;

img_bg_end_1 = img_bg_1[:,:,end]
w = size(img_bg_end_1, 1)
l = size(img_bg_end_1, 2)


# check how well they match
figure()
img_bg_end_1[chamber_roi.!=0].=NaN
imshow(img_bg_end_1', cmap="gray")


for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_1), "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n60.h5"))
x_bins = read(for_place_calculation_file,"x_bins")
y_bins = read(for_place_calculation_file,"y_bins")
close(for_place_calculation_file)

bin_interval = x_bins[2] - x_bins[1]

In [ ]:
chamber_roi_mask = Float32.(copy(chamber_roi))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32;


img_bg_mask = copy(img_bg_end_1)
img_bg_mask[img_bg_mask.==0].=NaN32
img_bg_mask[Float32.(chamber_roi).!=0].=NaN32;

In [ ]:
mask_valid = isfinite.(place_map_all_1[:,:,1])

mask_valid_index = findall(mask_valid)
mask_valid_x = [x[1] for x in mask_valid_index]
mask_valid_y = [x[2] for x in mask_valid_index];
mask_valid_x_min = minimum(mask_valid_x)
mask_valid_x_max = maximum(mask_valid_x)
mask_valid_y_min = minimum(mask_valid_y)
mask_valid_y_max = maximum(mask_valid_y);

valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

In [ ]:
roi_index = [123548, 120581, 105046, 79454, 119921, 81175, 94734, 118351, 94111, 72859, 98607, 109394, 121564, 123902, 116128, 94677, 123289, 128125, 116553, 59377, 103032, 108695, 99953, 120936, 126219, 80304, 88520, 90245, 73250, 88533]

In [ ]:
chosen_neuron = Int32.(neuron_label[roi_index])

In [ ]:
println(chosen_neuron)

In [ ]:
chosen_neuron = [71809, 37379, 65532, 10120, 64557, 20788, 43392, 2456, 28139, 23027, 62722, 59388, 53575, 37387, 65037, 34019, 68630, 40374, 6710, 47732, 48338, 14660, 45458, 50397, 63280, 73637, 63397, 56250, 9097, 14234]

In [ ]:
numpy.unique(chosen_neuron)

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure1/corner_cue"

In [ ]:
which_neuron = chosen_neuron

which_map = place_map_all_1[:,:,which_neuron];

fig = figure()

for i in 1:length(chosen_neuron)
    subplot(ceil(Int64, size(which_map,3)/6), 6,i)
    imshow(which_map[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,i]', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower")
    # imshow(chamber_roi_mask', cmap="binary")
    axis("off")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
    title(chosen_neuron[i])
end


# fig.savefig(joinpath(fig_path, "edge_confined_place_cells.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)

In [ ]:
which_neuron = chosen_neuron

which_map = place_map_all_1[:,:,which_neuron];

fig = figure(dpi=3000)

for i in 1:length(chosen_neuron)
    subplot(ceil(Int64, size(which_map,3)/6), 6,i)
    imshow(which_map[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,i]', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower")
    imshow(chamber_roi_mask', cmap="binary")
    axis("off")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
    # title(chosen_neuron[i])
end
subplots_adjust(left = 0, top = 0.1, right = 0.2, bottom = 0)


fig.savefig(joinpath(fig_path, "place_cells_different_locs.png"), bbox_inches = "tight",transparent = true)

# Look for alternatives

In [ ]:
# to find neurons with confined place fields (return index of place_cell_index) and the best peak of all place_cell_index
function neuron_with_valid_peak(place_cell_index, place_map_all;threshold=8/10, components_size_threshold = 10)
    mask_valid = isfinite.(place_map_all[:,:,1])
    best_peak_all = fill(NaN32, 2, length(place_cell_index))
    firing_field_size_ratio_all = fill(NaN32, length(place_cell_index))
    nr_modes = fill(NaN32, length(place_cell_index))
    @showprogress for (i, which_neuron) in enumerate(place_cell_index)
        which_map = place_map_all[:,:,which_neuron]
        components_peaks, img_label_valid, valid_components = map_components_peak(which_map; threshold = threshold, components_size_threshold = components_size_threshold)
        firing_field_size_ratio = sum(img_label_valid.!=0)/sum(mask_valid)
        peak_values = [numpy.nanpercentile(which_map[img_label_valid .== valid_components[i]], 95) for i in 1:length(valid_components)]
        if length(peak_values) .> 0
            best_peak = components_peaks[findmax(peak_values)[2]]
            firing_field_size_ratio_all[i] = firing_field_size_ratio
            best_peak_all[:,i] .= best_peak
            nr_modes[i] = length(valid_components)
        end
    end
    
    which_neuron = intersect(findall(firing_field_size_ratio_all.<0.2))
    confined_place_cell_index = place_cell_index[which_neuron];
    
    return which_neuron, best_peak_all, nr_modes
end

In [ ]:
which_neuron_1, peak_all_1, nr_modes_1 = neuron_with_valid_peak(tel_place_cell_index, place_map_all_1)
which_neuron = intersect(which_neuron_1, findall(nr_modes_1.==1))
confined_place_cell_index = tel_place_cell_index[which_neuron]

peak_loc_map_1 = peak_all_1[:,which_neuron] #peaks for cells with confined place fields

In [ ]:
# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
countour_array_1 = read(orientation_correction_file,"countour_array")
center_loc_1 = read(orientation_correction_file,"center_loc")
close(orientation_correction_file)


img_bg_1 = h5open(ds_save_1, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;

img_bg_end_1 = img_bg_1[:,:,end]
w = size(img_bg_end_1, 1)
l = size(img_bg_end_1, 2)


# check how well they match
figure()
img_bg_end_1[chamber_roi.!=0].=NaN
imshow(img_bg_end_1', cmap="gray")


for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_1), "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n60.h5"))
x_bins = read(for_place_calculation_file,"x_bins")
y_bins = read(for_place_calculation_file,"y_bins")
close(for_place_calculation_file)

bin_interval = x_bins[2] - x_bins[1]

In [ ]:
KMeans_features = cluster.KMeans(n_clusters = 6).fit(hcat(peak_loc_map_1'))
class_1 = KMeans_features.labels_;

In [ ]:
fig = figure()
scatter((peak_loc_map_1[1,:].-0.5).*bin_interval, (peak_loc_map_1[2,:].-0.5).*bin_interval, c=class_1, cmap="Dark2", s=10, alpha=0.3)
colorbar()
imshow(img_bg_end_1', cmap="binary", vmax=2000)

axis("off")

In [ ]:
which_neuron = confined_place_cell_index[class_1.==0][sortperm(peak_loc_map_1[2,class_1.==0])][1:10]

In [ ]:
# which_neuron = numpy.random.choice(confined_place_cell_index[class_1.==0], 50, replace=false)
which_map = place_map_all_1[:,:,which_neuron];

figure(figsize = (40,5*ceil(Int64, size(which_map,3)/10)), dpi=250)

for i in 1:size(which_map,3)
    subplot(ceil(Int64, size(which_map,3)/10), 10,i)
    imshow(which_map[:,:,i]') 
    axis("off")
    cell_index = which_neuron[i]
    title("$cell_index")
    peak = peak_loc_map_1[:,confined_place_cell_index.==cell_index]
    scatter(peak[1], peak[2], color="r")
    
end
-

In [ ]:
NMF_filename = joinpath(data_path(ds_save_1), "NMF.h5")
NMF_file = h5open(NMF_filename, "r")
global Z_all_roi = HDF5.readmmap(NMF_file["z_all"])
global X_all_roi = HDF5.readmmap(NMF_file["centroid_x_all"])
global Y_all_roi = HDF5.readmmap(NMF_file["centroid_y_all"])

close(NMF_file)


In [ ]:
findall(neuron_label.== 10120.0f0)

In [ ]:
plot_loc([79454, 72859]; X_all = X_all_roi,  Y_all = Y_all_roi,  Z_all = Z_all_roi, valid_neurons =1:length(Z_all_roi))